In [6]:
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt

"""
Load AirQuality UCI Data
"""

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './ds_1122_pr-data.csv'

df = pd.read_csv(input_file, index_col=[0], parse_dates=[0], date_parser=parser)

df.head()

<ipython-input-6-0a95f40b9227>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


,CO(GT),PT08.S1(CO),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH,C6H6(GT)
Datetime,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,1046.0,166.0,1056.0,113.0,1692.0,1268.0,48.9,0.7578,11.9
2004-03-10 19:00:00,2.0,1292.0,955.0,103.0,1174.0,92.0,1559.0,972.0,47.7,0.7255,9.4
2004-03-10 20:00:00,2.2,1402.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,54.0,0.7502,9.0
2004-03-10 21:00:00,2.2,1376.0,948.0,172.0,1092.0,122.0,1584.0,1203.0,60.0,0.7867,9.2
2004-03-10 22:00:00,1.6,1272.0,836.0,131.0,1205.0,116.0,1490.0,1110.0,59.6,0.7888,6.5


In [7]:
# Visualization setup
%matplotlib
%config InlineBackend.figure_format = 'svg'

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() # enable the interactive mode

import seaborn as sns
sns.set() # set plot styles

Using matplotlib backend: Qt5Agg


In [8]:
# Interpolate 'CO(GT)' column
df['CO(GT)'].interpolate(inplace=True)
df['CO(GT)'].plot()

<AxesSubplot:xlabel='Datetime'>

In [10]:
"""
Binning
"""

max_val = df['CO(GT)'].max()
min_val = df['CO(GT)'].min()
print(max_val, min_val)

11.9 0.0


In [11]:
# Make interval values
bins = np.linspace(min_val, max_val, 6)
bins

array([ 0.  ,  2.38,  4.76,  7.14,  9.52, 11.9 ])

In [18]:
# Labels for each bin

# labels = ['0<=x<2.38', '2.38<=x<4.76', '4.76<=x<7.14', '7.14<=x<9.52', '9.52<=x<11.9']
labels = ['0', '1', '2', '3', '4']

In [19]:
# Convert the numerical values into the categorical values

df['bins'] = pd.cut(df['CO(GT)'], bins=bins, labels=labels, include_lowest=True)

df['CO(GT)']

Datetime
2004-03-10 18:00:00    2.6
2004-03-10 19:00:00    2.0
2004-03-10 20:00:00    2.2
2004-03-10 21:00:00    2.2
2004-03-10 22:00:00    1.6
                      ... 
2005-04-04 10:00:00    3.1
2005-04-04 11:00:00    2.4
2005-04-04 12:00:00    2.4
2005-04-04 13:00:00    2.1
2005-04-04 14:00:00    2.2
Name: CO(GT), Length: 9357, dtype: float64

In [20]:
# Print bins
df['bins']

Datetime
2004-03-10 18:00:00    1
2004-03-10 19:00:00    0
2004-03-10 20:00:00    0
2004-03-10 21:00:00    0
2004-03-10 22:00:00    0
                      ..
2005-04-04 10:00:00    1
2005-04-04 11:00:00    1
2005-04-04 12:00:00    1
2005-04-04 13:00:00    0
2005-04-04 14:00:00    0
Name: bins, Length: 9357, dtype: category
Categories (5, object): ['0' < '1' < '2' < '3' < '4']

In [21]:
# Visualize the histogram of bins
plt.hist(df['bins'], bins=5)
plt.show()

In [24]:
"""
Log Transform
"""

# Distribution of original data

# sns.distplot(df['PT08.S3(NOx)'])
df['PT08.S3(NOx)'].plot()


<AxesSubplot:xlabel='Datetime'>

In [25]:
# Calculate natural logarithm on 'CO(GT)' column
df['log'] = np.log10(df['PT08.S3(NOx)']) # log10: common logarithm, 상용로그
df['log'].plot()

<AxesSubplot:xlabel='Datetime'>

In [27]:
# Min values for each column
df.min()

CO(GT)                0.0
PT08.S1(CO)         647.0
PT08.S2(NMHC)       383.0
NOx(GT)               2.0
PT08.S3(NOx)        322.0
NO2(GT)               2.0
PT08.S4(NO2)        551.0
PT08.S5(O3)         221.0
RH                    9.2
AH                 0.1847
C6H6(GT)              0.0
bins                    0
log              2.507856
dtype: object

In [28]:
# Distribution after log transform
sns.distplot(df['log'])
plt.xlabel('log(NoX)')
plt.show()

E:\Anaconda\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [29]:
"""
One-hot Encoding
"""

# Make a dataset

emp_id = pd.Series([1, 2, 3, 4, 5])
gender = pd.Series(['Male', 'Female', 'Female', 'Male', 'Female'])
remarks = pd.Series(['Nice', 'Good', 'Great', 'Great', 'Nice'])

df_emp = pd.DataFrame()
df_emp['emp_id'] = emp_id
df_emp['gender'] = gender
df_emp['remarks'] = remarks

df_emp

,emp_id,gender,remarks
0,1,Male,Nice
1,2,Female,Good
2,3,Female,Great
3,4,Male,Great
4,5,Female,Nice


In [32]:
# Print unique values for each column
print(df_emp['emp_id'].unique())
print(df_emp['gender'].unique())
print(df_emp['remarks'].unique())

[1 2 3 4 5]
['Male' 'Female']
['Nice' 'Good' 'Great']


In [34]:
# One-hot encoding the categorial values
df_emp_encoded = pd.get_dummies(df_emp, columns=['gender', 'remarks'])
df_emp_encoded

,emp_id,gender_Female,gender_Male,remarks_Good,remarks_Great,remarks_Nice
0,1,0,1,0,0,1
1,2,1,0,1,0,0
2,3,1,0,0,1,0
3,4,0,1,0,1,0
4,5,1,0,0,0,1


In [35]:
"""
Normalization
"""

# Visualize two columns of different scales
plt.plot(df['CO(GT)'], label='CO')
plt.plot(df['PT08.S2(NMHC)'], label='NMHC')
plt.legend(loc='best')

In [37]:
# Normalize the 'CO(GT)' column

co = df['CO(GT)'].copy()
co_max = co.max()
co_min = co.min()

df['CO_Norm'] = (co - co_min) / (co_max - co_min)
df['CO_Norm']

Datetime
2004-03-10 18:00:00    0.218487
2004-03-10 19:00:00    0.168067
2004-03-10 20:00:00    0.184874
2004-03-10 21:00:00    0.184874
2004-03-10 22:00:00    0.134454
                         ...   
2005-04-04 10:00:00    0.260504
2005-04-04 11:00:00    0.201681
2005-04-04 12:00:00    0.201681
2005-04-04 13:00:00    0.176471
2005-04-04 14:00:00    0.184874
Name: CO_Norm, Length: 9357, dtype: float64

In [39]:
# Normalize the 'PT08.S2(NMHC)' column
nmhc = df['PT08.S2(NMHC)'].copy()
nmhc_max = nmhc.max()
nmhc_min = nmhc.min()

df['NMHC_Norm'] = (nmhc - nmhc_min) / (nmhc_max - nmhc_min)
df['NMHC_Norm']

Datetime
2004-03-10 18:00:00    0.362097
2004-03-10 19:00:00    0.312398
2004-03-10 20:00:00    0.303659
2004-03-10 21:00:00    0.308575
2004-03-10 22:00:00    0.247406
                         ...   
2005-04-04 10:00:00    0.392135
2005-04-04 11:00:00    0.351720
2005-04-04 12:00:00    0.371382
2005-04-04 13:00:00    0.315674
2005-04-04 14:00:00    0.362643
Name: NMHC_Norm, Length: 9357, dtype: float64

In [41]:
# Visualized normalized columns
plt.plot(df['CO_Norm'], label='CO (normalized)')
plt.plot(df['NMHC_Norm'], label='NMHC (normalized)')
plt.legend(loc='best')

In [42]:
"""
Feature Split
"""

# Make untidy movie data
movies = pd.Series(["The Godfather, 1972, Francis Ford Coppola", "Contact, 1997, Robert Zemeckis", "Parasite, 2019, Joon-ho Bong"])

movies

0    The Godfather, 1972, Francis Ford Coppola
1               Contact, 1997, Robert Zemeckis
2                 Parasite, 2019, Joon-ho Bong
dtype: object

In [43]:
# Divide movie data into title, year, director columns
lst_title = []
lst_year = []
lst_director = []

for val in movies:
    title, year, director = val.split(',') # data split
    lst_title.append(title)
    lst_year.append(year)
    lst_director.append(director)

print(lst_title)
print(lst_year)
print(lst_director)

['The Godfather', 'Contact', 'Parasite']
[' 1972', ' 1997', ' 2019']
[' Francis Ford Coppola', ' Robert Zemeckis', ' Joon-ho Bong']


In [46]:
# Make a DataFrame object
df_movie = pd.DataFrame()
df_movie['title'] = lst_title
df_movie['year'] = lst_year
df_movie['director'] = lst_director

df_movie

,title,year,director
0,The Godfather,1972,Francis Ford Coppola
1,Contact,1997,Robert Zemeckis
2,Parasite,2019,Joon-ho Bong
